# AOC 2025 Day 12

In [1]:
@file:DependsOn("com.toldoven.aoc:aoc-kotlin-notebook:1.1.2")

In [2]:
import com.toldoven.aoc.notebook.AocClient

val aocClient = AocClient.fromEnv().interactiveDay(2025, 12)
val input = aocClient.input()
aocClient.viewPartOne()

--- Day 12: Christmas Tree Farm --- 
 You're almost out of time, but there can't be much left to decorate. Although there are no stairs, elevators, escalators, tunnels, chutes, teleporters, firepoles, or conduits here that would take you deeper into the North Pole base, there is a ventilation duct. You jump in. 
 After bumping around for a few minutes, you emerge into a large, well-lit cavern full of Christmas trees! 
 There are a few Elves here frantically decorating before the deadline. They think they'll be able to finish most of the work, but the one thing they're worried about is the presents for all the young Elves that live here at the North Pole. It's an ancient tradition to put the presents under the trees, but the Elves are worried they won't fit . 
 The presents come in a few standard but very weird shapes. The shapes and the regions into which they need to fit are all measured in standard units . To be aesthetically pleasing, the presents need to be placed into the regions in a way that follows a standardized two-dimensional unit grid; you also can't stack presents. 
 As always, the Elves have a summary of the situation (your puzzle input) for you. First, it contains a list of the presents' shapes. Second, it contains the size of the region under each tree and a list of the number of presents of each shape that need to fit into that region. For example: 
 0:
###
##.
##.

1:
###
##.
.##

2:
.##
###
##.

3:
##.
###
##.

4:
###
#..
###

5:
###
.#.
###

4x4: 0 0 0 0 2 0
12x5: 1 0 1 0 2 2
12x5: 1 0 1 0 3 2
 
 The first section lists the standard present shapes . For convenience, each shape starts with its index and a colon; then, the shape is displayed visually, where # is part of the shape and . is not. 
 The second section lists the regions under the trees. Each line starts with the width and length of the region; 12x5 means the region is 12 units wide and 5 units long. The rest of the line describes the presents that need to fit into that region by listing the quantity of each shape of present; 1 0 1 0 3 2 means you need to fit one present with shape index 0, no presents with shape index 1, one present with shape index 2, no presents with shape index 3, three presents with shape index 4, and two presents with shape index 5. 
 Presents can be rotated and flipped as necessary to make them fit in the available space, but they have to always be placed perfectly on the grid. Shapes can't overlap (that is, the # part from two different presents can't go in the same place on the grid), but they can fit together (that is, the . part in a present's shape's diagram does not block another present from occupying that space on the grid). 
 The Elves need to know how many of the regions can fit the presents listed. In the above example, there are six unique present shapes and three regions that need checking. 
 The first region is 4x4: 
 ....
....
....
....
 
 In it, you need to determine whether you could fit two presents that have shape index 4 : 
 ###
#..
###
 
 After some experimentation, it turns out that you can fit both presents in this region. Here is one way to do it, using A to represent one present and B to represent the other: 
 AAA.
ABAB
ABAB
.BBB
 
 The second region, 12x5: 1 0 1 0 2 2 , is 12 units wide and 5 units long. In that region, you need to try to fit one present with shape index 0 , one present with shape index 2 , two presents with shape index 4 , and two presents with shape index 5 . 
 It turns out that these presents can all fit in this region. Here is one way to do it, again using different capital letters to represent all the required presents: 
 ....AAAFFE.E
.BBBAAFFFEEE
DDDBAAFFCECE
DBBB....CCC.
DDD.....C.C.
 
 The third region, 12x5: 1 0 1 0 3 2 , is the same size as the previous region; the only difference is that this region needs to fit one additional present with shape index 4 . Unfortunately, no matter how hard you try, there is no way to fit all of the presents into this region. 
 So, in this 

In [4]:
data class Point(val x: Int, val y: Int)

// pre-calculated shape variant
data class Variant(val width: Int, val height: Int, val points: Set<Point>)

data class RegionTask(val width: Int, val height: Int, val presentIDs: List<Int>)

// Geometry helpers
fun generateVariants(basePoints: Set<Point>): List<Variant> {
    val variants = mutableSetOf<Set<Point>>()
    var current = basePoints

    // generate 4 rotations and their mirrored versions
    repeat(4) {
        variants.add(normalize(current))
        variants.add(normalize(flip(current)))
        current = rotate90(current)
    }

    return variants.map { pts ->
        val w = (pts.maxOfOrNull { it.x } ?: 0) + 1
        val h = (pts.maxOfOrNull { it.y } ?: 0) + 1
        Variant(w, h, pts)
    }
}

fun rotate90(points: Set<Point>) = points.map { Point(-it.y, it.x) }.toSet()
fun flip(points: Set<Point>) = points.map { Point(-it.x, it.y) }.toSet()
fun normalize(points: Set<Point>): Set<Point> {
    if (points.isEmpty()) return points
    val minX = points.minOf { it.x }
    val minY = points.minOf { it.y }
    return points.map { Point(it.x - minX, it.y - minY) }.toSet()
}

// Solving Logic
fun placeRecursively(
    index: Int,
    presents: kotlin.collections.List<Pair<Int, kotlin.collections.List<Variant>>>,
    grid: BooleanArray,
    boardW: Int,
    boardH: Int
): Boolean {
    if (index == presents.size) return true

    val (_, variants) = presents[index]

    // Try all orientations (variants) of the current present
    for (variant in variants) {
        // Skip if variant dimensions exceed board
        if (variant.width > boardW || variant.height > boardH) continue

        // Try to place at every possible top-left position (i)
        for (i in grid.indices) {
            // Optimization: If the starting cell is already occupied, we can't place the top-left of the shape here.
            // Note: This assumes strict packing logic where shapes are solid.
            // Since shapes can have holes ('.'), strictly speaking, the top-left of the BOUNDING BOX
            // could be over an occupied cell if the shape itself has a hole there.
            // However, checking grid[i] is a very good heuristic for dense packing.
            // If strictly 'sparse' shapes are allowed where 0,0 is empty, remove `if (grid[i]) continue`.
            // Given AoC style, let's play it safe and check the full shape fit.

            val x = i % boardW
            val y = i / boardW

            // Boundary check for the bounding box
            if (x + variant.width > boardW || y + variant.height > boardH) continue

            if (canPlace(grid, variant, x, y, boardW)) {
                // Place
                toggleGrid(grid, variant, x, y, boardW, true)

                // Recurse
                if (placeRecursively(index + 1, presents, grid, boardW, boardH)) return true

                // Backtrack (Remove)
                toggleGrid(grid, variant, x, y, boardW, false)
            }
        }
    }
    return false
}

fun canPlace(grid: BooleanArray, variant: Variant, x: Int, y: Int, boardW: Int): Boolean {
    for (p in variant.points) {
        // Since we checked bounding box in the loop before, we only need to check collision
        if (grid[(y + p.y) * boardW + (x + p.x)]) return false
    }
    return true
}

fun toggleGrid(grid: BooleanArray, variant: Variant, x: Int, y: Int, boardW: Int, state: Boolean) {
    for (p in variant.points) {
        grid[(y + p.y) * boardW + (x + p.x)] = state
    }
}


fun solveRegion(task: RegionTask, shapeMap: Map<Int, List<Variant>>): Boolean {
    // Optimization: Sort presents by area descending (largest first -> fail fast)
    val sortedPresents = task.presentIDs.map { id ->
        id to (shapeMap[id] ?: error("Unknown shape $id"))
    }.sortedByDescending { (_, variants) -> variants.first().points.size }

    // Quick check: Does total present area fit in grid area?
    val totalArea = sortedPresents.sumOf { (_, v) -> v.first().points.size }
    if (totalArea > task.width * task.height) return false

    val grid = BooleanArray(task.width * task.height) { false }
    return placeRecursively(0, sortedPresents, grid, task.width, task.height)
}

// Solving the thing
val parts = input.trim().split("\n\n")
val shapeMap = mutableMapOf<Int, List<Variant>>()
val regions = mutableListOf<RegionTask>()

// parse mixed blocks
parts.forEach { block ->
    if (block.lines().any {it.contains("x") && it.contains(":")}) {
        // It's a task/region block
        block.lines().forEach { line ->
            if (!line.contains(":")) return@forEach
            val (dimPart, presentsPart) = line.split(": ")
            val (w, h) = dimPart.split("x").map { it.toInt() }
            val counts = presentsPart.trim().split(" ").map { it.toInt() }

            val ids = mutableListOf<Int>()
            counts.forEachIndexed { id, count -> repeat(count) { ids.add(id) } }
            regions.add(RegionTask(w, h, ids))
        }
    } else {
        // it's a shape definition
        val lines = block.lines()
        val id = lines.first().substringBefore(":").toInt()
        val points = mutableSetOf<Point>()
        lines.drop(1).forEachIndexed { y, row ->
            row.forEachIndexed { x, char ->
                if (char == '#') points.add(Point(x, y))
            }
        }
        shapeMap[id] = generateVariants(points)
    }
}

println("Parsed ${shapeMap.size} shapes and ${regions.size} regions.")

val answerPt1 = regions.count { solveRegion(it, shapeMap) }
aocClient.submitPartOne(answerPt1)

Parsed 6 shapes and 1000 regions.


Your answer: 414. 
 That's the right answer! You are one gold star closer to decorating the North Pole. [Continue to Part Two]

In [7]:
aocClient.viewPartTwo()

--- Part Two --- 
 The Elves thank you profusely for the help and start rearranging the oddly-shaped presents. As you look up, you notice that a lot more Elves have arrived here at the Christmas tree farm. 
 In fact, many of these new arrivals look familiar : they're the Elves you helped while decorating the North Pole base. Right on schedule , each group seems to have brought a star to put atop one of the Christmas trees! 
 Before any of them can find a ladder, a particularly large Christmas tree suddenly flashes brightly when a large star magically appears above it! As your eyes readjust, you think you notice a portly man with a white beard disappear into the crowd. 
 You go look for a ladder; only 23 stars to go.